In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict

logging.basicConfig(level=logging.INFO)

In [3]:
dataset = "mnist"
algo = "monolithic"

experiment_folder = "experiment_results"
experiment_name = "vanilla_jorge_setting_basis__no_sparse_basis_False"

use_contrastive = True
# use_contrastive = False
num_trains_per_class = 64
seed = 0
num_tasks = 10
parallel = False
comm_freq = 2 # "None" means no communication, doesn't matter for this analysis


save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)


In [4]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    return net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [5]:
# fleet = setup_fleet(save_dir=save_dir, task_id=3, parallel=False)
# fleet = setup_fleet(save_dir=save_dir, parallel=False)
fleet = setup_fleet(save_dir=save_dir, parallel=True)

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': False}, 'sharing_strategy': {'name': 'no_sharing', 'num_coms_per_round': 0, 'sync_base': False, 'pre_or_post_comm': 'post'}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic_numtrain_64_contrastive', 'num_agents': 8, 'root_save_dir': 'experiment_results/vanilla_jorge_setting_basis__no_sparse_basis_False', 'parallel': True, 'num_init_tasks': 4, 'overwrite': False, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': True, 'use_ood_separation

INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28)
INFO:root:Class sequence: [6 7 7 8 4 1 1 8 6 1 6 4 5 7 8 0 2 3 0 3]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28)
INFO:root:task 8 :(128, 

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': False, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': True}
<class 'shell.learners.er_nocomponents.NoComponentsER'>


2024-03-01 14:30:07,103	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ParallelAgent pid=2905232) INFO:root:Agent: node_id: 0, seed: 0
(ParallelAgent pid=2905233) INFO:root:Agent: node_id: 1, seed: 1000
(ParallelAgent pid=2905234) INFO:root:Agent: node_id: 2, seed: 2000
(ParallelAgent pid=2905271) INFO:root:Agent: node_id: 3, seed: 3000
(ParallelAgent pid=2905335) INFO:root:Agent: node_id: 4, seed: 4000
(ParallelAgent pid=2905413) INFO:root:Agent: node_id: 5, seed: 5000
(ParallelAgent pid=2905482) INFO:root:Agent: node_id: 6, seed: 6000
(ParallelAgent pid=2905543) INFO:root:Agent: node_id: 7, seed: 7000
INFO:root:Created fleet with 8 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


In [6]:
for i in range(4):
    fleet.train_and_comm(task_id=i)

(ParallelAgent pid=2905233) agent train 0 0 101 True {}
(ParallelAgent pid=2905232) agent train 0 0 101 True {}
(ParallelAgent pid=2905234) agent train 0 0 101 True {}
(ParallelAgent pid=2905271) agent train 0 0 101 True {}
(ParallelAgent pid=2905335) agent train 0 0 101 True {}
(ParallelAgent pid=2905413) agent train 0 0 101 True {}
(ParallelAgent pid=2905482) agent train 0 0 101 True {}
(ParallelAgent pid=2905543) agent train 0 0 101 True {}


(ParallelAgent pid=2905234) INFO:root:epochs: 0, training task: 0
(ParallelAgent pid=2905234) INFO:root:	task: 0	loss: 0.69338607	acc: 0.518
(ParallelAgent pid=2905234) INFO:root:	task: avg	loss: 0.69338607	acc: 0.518
(ParallelAgent pid=2905335) INFO:root:epochs: 0, training task: 0
(ParallelAgent pid=2905335) INFO:root:	task: 0	loss: 0.69287323	acc: 0.508
(ParallelAgent pid=2905335) INFO:root:	task: avg	loss: 0.69287323	acc: 0.508
(ParallelAgent pid=2905232) INFO:root:epochs: 0, training task: 0
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.69148300	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: avg	loss: 0.69148300	acc: 0.524
(ParallelAgent pid=2905233) INFO:root:epochs: 0, training task: 0
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69484174	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: avg	loss: 0.69484174	acc: 0.511
(ParallelAgent pid=2905271) INFO:root:epochs: 0, training task: 0
(ParallelAgent pid=2905271) INFO:root:	task: 0	loss: 0.6981669

(ParallelAgent pid=2905233) agent train 1 0 101 True {}
(ParallelAgent pid=2905232) agent train 1 0 101 True {}
(ParallelAgent pid=2905234) agent train 1 0 101 True {}
(ParallelAgent pid=2905271) agent train 1 0 101 True {}
(ParallelAgent pid=2905335) agent train 1 0 101 True {}
(ParallelAgent pid=2905413) agent train 1 0 101 True {}
(ParallelAgent pid=2905482) agent train 1 0 101 True {}
(ParallelAgent pid=2905543) agent train 1 0 101 True {}


(ParallelAgent pid=2905232) INFO:root:epochs: 0, training task: 1
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.69148300	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: 1	loss: 0.69276926	acc: 0.507
(ParallelAgent pid=2905232) INFO:root:	task: avg	loss: 0.69212613	acc: 0.515
(ParallelAgent pid=2905233) INFO:root:epochs: 0, training task: 1
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69484174	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: 1	loss: 0.69288743	acc: 0.536
(ParallelAgent pid=2905233) INFO:root:	task: avg	loss: 0.69386459	acc: 0.524
(ParallelAgent pid=2905234) INFO:root:epochs: 0, training task: 1
(ParallelAgent pid=2905234) INFO:root:	task: 0	loss: 0.69338607	acc: 0.518
(ParallelAgent pid=2905234) INFO:root:	task: 1	loss: 0.69270822	acc: 0.513
(ParallelAgent pid=2905234) INFO:root:	task: avg	loss: 0.69304715	acc: 0.516
(ParallelAgent pid=2905271) INFO:root:epochs: 0, training task: 1
(ParallelAgent pid=2905271) INFO:root:	task: 0	loss: 0.

(ParallelAgent pid=2905233) agent train 2 0 101 True {}
(ParallelAgent pid=2905232) agent train 2 0 101 True {}
(ParallelAgent pid=2905234) agent train 2 0 101 True {}
(ParallelAgent pid=2905271) agent train 2 0 101 True {}
(ParallelAgent pid=2905335) agent train 2 0 101 True {}
(ParallelAgent pid=2905413) agent train 2 0 101 True {}
(ParallelAgent pid=2905482) agent train 2 0 101 True {}
(ParallelAgent pid=2905543) agent train 2 0 101 True {}


(ParallelAgent pid=2905233) INFO:root:epochs: 0, training task: 2
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69484174	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: 1	loss: 0.69288743	acc: 0.536
(ParallelAgent pid=2905233) INFO:root:	task: 2	loss: 0.69363158	acc: 0.506
(ParallelAgent pid=2905233) INFO:root:	task: avg	loss: 0.69378692	acc: 0.518
(ParallelAgent pid=2905234) INFO:root:epochs: 0, training task: 2
(ParallelAgent pid=2905234) INFO:root:	task: 0	loss: 0.69338607	acc: 0.518
(ParallelAgent pid=2905234) INFO:root:	task: 1	loss: 0.69270822	acc: 0.513
(ParallelAgent pid=2905234) INFO:root:	task: 2	loss: 0.69485889	acc: 0.464
(ParallelAgent pid=2905234) INFO:root:	task: avg	loss: 0.69365106	acc: 0.498
(ParallelAgent pid=2905271) INFO:root:epochs: 0, training task: 2
(ParallelAgent pid=2905271) INFO:root:	task: 0	loss: 0.69816695	acc: 0.486
(ParallelAgent pid=2905271) INFO:root:	task: 1	loss: 0.69343265	acc: 0.398
(ParallelAgent pid=2905271) INFO:root:	task: 2	l

(ParallelAgent pid=2905233) agent train 3 0 101 True {}
(ParallelAgent pid=2905232) agent train 3 0 101 True {}
(ParallelAgent pid=2905234) agent train 3 0 101 True {}
(ParallelAgent pid=2905271) agent train 3 0 101 True {}
(ParallelAgent pid=2905335) agent train 3 0 101 True {}
(ParallelAgent pid=2905413) agent train 3 0 101 True {}
(ParallelAgent pid=2905482) agent train 3 0 101 True {}
(ParallelAgent pid=2905543) agent train 3 0 101 True {}


(ParallelAgent pid=2905232) INFO:root:epochs: 0, training task: 3
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.69148300	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: 1	loss: 0.69276926	acc: 0.507
(ParallelAgent pid=2905232) INFO:root:	task: 2	loss: 0.69821708	acc: 0.494
(ParallelAgent pid=2905232) INFO:root:	task: 3	loss: 0.69508529	acc: 0.493
(ParallelAgent pid=2905232) INFO:root:	task: avg	loss: 0.69438866	acc: 0.504
(ParallelAgent pid=2905233) INFO:root:epochs: 0, training task: 3
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69484174	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: 1	loss: 0.69288743	acc: 0.536
(ParallelAgent pid=2905233) INFO:root:	task: 2	loss: 0.69363158	acc: 0.506
(ParallelAgent pid=2905233) INFO:root:	task: 3	loss: 0.69278995	acc: 0.508
(ParallelAgent pid=2905233) INFO:root:	task: avg	loss: 0.69353767	acc: 0.515
(ParallelAgent pid=2905234) INFO:root:epochs: 0, training task: 3
(ParallelAgent pid=2905234) INFO:root:	task: 0	l

In [7]:
fleet.train_and_comm(task_id=4)

(ParallelAgent pid=2905233) agent train 4 0 101 True {}
(ParallelAgent pid=2905232) agent train 4 0 101 True {}
(ParallelAgent pid=2905234) agent train 4 0 101 True {}
(ParallelAgent pid=2905271) agent train 4 0 101 True {}
(ParallelAgent pid=2905335) agent train 4 0 101 True {}
(ParallelAgent pid=2905413) agent train 4 0 101 True {}
(ParallelAgent pid=2905482) agent train 4 0 101 True {}
(ParallelAgent pid=2905543) agent train 4 0 101 True {}


(ParallelAgent pid=2905232) INFO:root:epochs: 0, training task: 4
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.69360220	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: 1	loss: 0.69308404	acc: 0.507
(ParallelAgent pid=2905232) INFO:root:	task: 2	loss: 0.69377382	acc: 0.494
(ParallelAgent pid=2905232) INFO:root:	task: 3	loss: 0.69323044	acc: 0.493
(ParallelAgent pid=2905232) INFO:root:	task: 4	loss: 0.69323656	acc: 0.498
(ParallelAgent pid=2905232) INFO:root:	task: avg	loss: 0.69338541	acc: 0.503
(ParallelAgent pid=2905233) INFO:root:epochs: 0, training task: 4
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69334497	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: 1	loss: 0.69146886	acc: 0.535
(ParallelAgent pid=2905233) INFO:root:	task: 2	loss: 0.69341099	acc: 0.494
(ParallelAgent pid=2905233) INFO:root:	task: 3	loss: 0.69427823	acc: 0.508
(ParallelAgent pid=2905233) INFO:root:	task: 4	loss: 0.69300620	acc: 0.524
(ParallelAgent pid=2905233) INFO:root:	ta

(ParallelAgent pid=2905233) mega_dataset 256
(ParallelAgent pid=2905233) epoch: 0
(ParallelAgent pid=2905232) mega_dataset 256
(ParallelAgent pid=2905232) epoch: 0
(ParallelAgent pid=2905234) mega_dataset 256
(ParallelAgent pid=2905234) epoch: 0
(ParallelAgent pid=2905271) mega_dataset 256
(ParallelAgent pid=2905271) epoch: 0
(ParallelAgent pid=2905335) mega_dataset 256
(ParallelAgent pid=2905335) epoch: 0
(ParallelAgent pid=2905482) mega_dataset 256
(ParallelAgent pid=2905482) epoch: 0
(ParallelAgent pid=2905543) mega_dataset 256
(ParallelAgent pid=2905543) epoch: 0
(ParallelAgent pid=2905413) mega_dataset 256
(ParallelAgent pid=2905413) epoch: 0


(ParallelAgent pid=2905413) INFO:root:epochs: 0, training task: 4
(ParallelAgent pid=2905413) INFO:root:	task: 0	loss: 0.69322744	acc: 0.498
(ParallelAgent pid=2905413) INFO:root:	task: 1	loss: 0.69335469	acc: 0.487
(ParallelAgent pid=2905413) INFO:root:	task: 2	loss: 0.69168914	acc: 0.529
(ParallelAgent pid=2905413) INFO:root:	task: 3	loss: 0.69672638	acc: 0.458
(ParallelAgent pid=2905413) INFO:root:	task: 4	loss: 0.69377099	acc: 0.493
(ParallelAgent pid=2905413) INFO:root:	task: avg	loss: 0.69375373	acc: 0.493
(ParallelAgent pid=2905232) INFO:root:epochs: 1, training task: 4
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.69361736	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: 1	loss: 0.69299885	acc: 0.507
(ParallelAgent pid=2905232) INFO:root:	task: 2	loss: 0.69376332	acc: 0.494
(ParallelAgent pid=2905232) INFO:root:	task: 3	loss: 0.69300207	acc: 0.493
(ParallelAgent pid=2905232) INFO:root:	task: 4	loss: 0.69320770	acc: 0.498
(ParallelAgent pid=2905232) INFO:root:	ta

(ParallelAgent pid=2905233) epoch: 1
(ParallelAgent pid=2905232) epoch: 1
(ParallelAgent pid=2905234) epoch: 1
(ParallelAgent pid=2905271) epoch: 1
(ParallelAgent pid=2905335) epoch: 1
(ParallelAgent pid=2905482) epoch: 1
(ParallelAgent pid=2905543) epoch: 1
(ParallelAgent pid=2905232) epoch: 2
(ParallelAgent pid=2905234) epoch: 2
(ParallelAgent pid=2905335) epoch: 2
(ParallelAgent pid=2905482) epoch: 2
(ParallelAgent pid=2905233) epoch: 2
(ParallelAgent pid=2905271) epoch: 2
(ParallelAgent pid=2905413) epoch: 1
(ParallelAgent pid=2905543) epoch: 2


...

(ParallelAgent pid=2905335) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905335) INFO:root:	task: 0	loss: 0.50705983	acc: 0.975
(ParallelAgent pid=2905335) INFO:root:	task: 1	loss: 0.49564485	acc: 0.964
(ParallelAgent pid=2905335) INFO:root:	task: 2	loss: 0.58890330	acc: 0.792
(ParallelAgent pid=2905335) INFO:root:	task: 3	loss: 0.50242184	acc: 0.975
(ParallelAgent pid=2905335) INFO:root:	task: 4	loss: 0.68887911	acc: 0.567
(ParallelAgent pid=2905335) INFO:root:	task: avg	loss: 0.55658179	acc: 0.855
(ParallelAgent pid=2905482) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905482) INFO:root:	task: 0	loss: 0.70228071	acc: 0.465
(ParallelAgent pid=2905482) INFO:root:	task: 1	loss: 0.69533012	acc: 0.502
(ParallelAgent pid=2905482) INFO:root:	task: 2	loss: 0.69580088	acc: 0.513
(ParallelAgent pid=2905482) INFO:root:	task: 3	loss: 0.69663453	acc: 0.509
(ParallelAgent pid=2905482) INFO:root:	task: 4	loss: 0.69626420	acc: 0.506
(ParallelAgent pid=2905482) INFO:root

(ParallelAgent pid=2905413) epoch: 98


(ParallelAgent pid=2905233) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905233) INFO:root:	task: 0	loss: 0.69289089	acc: 0.511
(ParallelAgent pid=2905233) INFO:root:	task: 1	loss: 0.70810885	acc: 0.465
(ParallelAgent pid=2905233) INFO:root:	task: 2	loss: 0.69317534	acc: 0.506
(ParallelAgent pid=2905233) INFO:root:	task: 3	loss: 0.69421541	acc: 0.508
(ParallelAgent pid=2905233) INFO:root:	task: 4	loss: 0.69743029	acc: 0.476
(ParallelAgent pid=2905233) INFO:root:	task: avg	loss: 0.69716416	acc: 0.493
(ParallelAgent pid=2905271) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905271) INFO:root:	task: 0	loss: 0.69543426	acc: 0.514
(ParallelAgent pid=2905271) INFO:root:	task: 1	loss: 0.69346820	acc: 0.492
(ParallelAgent pid=2905271) INFO:root:	task: 2	loss: 0.69505152	acc: 0.489
(ParallelAgent pid=2905271) INFO:root:	task: 3	loss: 0.69514169	acc: 0.493
(ParallelAgent pid=2905271) INFO:root:	task: 4	loss: 0.69205864	acc: 0.524
(ParallelAgent pid=2905271) INFO:root

(ParallelAgent pid=2905413) epoch: 99


(ParallelAgent pid=2905232) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905232) INFO:root:	task: 0	loss: 0.64885153	acc: 0.524
(ParallelAgent pid=2905232) INFO:root:	task: 1	loss: 0.64732439	acc: 0.821
(ParallelAgent pid=2905232) INFO:root:	task: 2	loss: 0.70479144	acc: 0.494
(ParallelAgent pid=2905232) INFO:root:	task: 3	loss: 0.64473931	acc: 0.504
(ParallelAgent pid=2905232) INFO:root:	task: 4	loss: 0.54825617	acc: 0.869
(ParallelAgent pid=2905232) INFO:root:	task: avg	loss: 0.63879257	acc: 0.642
(ParallelAgent pid=2905234) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905234) INFO:root:	task: 0	loss: 0.70411877	acc: 0.482
(ParallelAgent pid=2905234) INFO:root:	task: 1	loss: 0.70963941	acc: 0.487
(ParallelAgent pid=2905234) INFO:root:	task: 2	loss: 0.69636030	acc: 0.464
(ParallelAgent pid=2905234) INFO:root:	task: 3	loss: 0.69437489	acc: 0.462
(ParallelAgent pid=2905543) INFO:root:epochs: 100, training task: 4
(ParallelAgent pid=2905543) INFO:root:	task:

In [10]:
agent = fleet.agents[0]
agent.agent

AttributeError: 'ActorHandle' object has no attribute 'agent'

In [9]:
# agent.train(task_id=4)